In [1]:
!curl https://www.secrepo.com/maccdc2012/conn.log.gz | gzip -d > conn.log

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  523M  100  523M    0     0   560k      0  0:15:56  0:15:56 --:--:--  680kM    0     0   634k      0  0:14:04  0:00:37  0:13:27  525k2k      0  0:15:04  0:02:49  0:12:15  433k269M    0     0   561k      0  0:15:54  0:08:11  0:07:43  648k  0     0   562k      0  0:15:53  0:08:26  0:07:27  654k0:09:21  0:06:26  407k 0     0   565k      0  0:15:47  0:09:26  0:06:21  534k    0   568k      0  0:15:43  0:10:29  0:05:14  495k    0     0   565k      0  0:15:47  0:12:00  0:03:47  498k0     0   560k      0  0:15:57  0:15:30  0:00:27  474k


In [ ]:
### 20% sample. This avoids cudf OoMing on a GTX 1070

### If you OoM, you usually must restart the kernel & refresh the page

In [2]:
!awk 'NR % 20 == 0' < conn.log > conn-5pc.log

In [3]:
!awk 'NR % 5 == 0' < conn.log > conn-20pc.log

In [4]:
import pandas as pd
import cudf


In [5]:
!head -n 3 conn.log

1331901000.000000	CCUIP21wTjqkj8ZqX5	192.168.202.79	50463	192.168.229.251	80	tcp	-	-	-	-	SH	-	0	Fa	1	52	1	52	(empty)
1331901000.000000	Csssjd3tX0yOTPDpng	192.168.202.79	46117	192.168.229.254	443	tcp	-	-	-	-	SF	-	0	dDafFr	3	382	9	994	(empty)
1331901000.000000	CHEt7z3AzG4gyCNgci	192.168.202.79	50465	192.168.229.251	80	tcp	http	0.010000	166	214	SF	-	0	ShADfFa	4	382	3	382	(empty)


In [6]:
df_thunk = lambda: pd.read_csv("./conn-20pc.log", sep="\t", header=None, names=["time", "x0", "srcip", "srcport", "dstip", "dstport", "transport", "protocol", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10", "x11"], na_values=['-'], index_col=False)


In [7]:
%%timeit
df_thunk()

9.82 s ± 43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
df=df_thunk()

In [9]:
cudf_thunk = lambda: cudf.read_csv("./conn-20pc.log", sep="\t", header=None, names=["time", "x0", "srcip", "srcport", "dstip", "dstport", "transport", "protocol", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10","x11"], na_values=['-'], index_col=False)

In [10]:
%%timeit
cudf_thunk()

759 ms ± 5.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
cdf = cudf_thunk()

In [12]:
srcdstcount_thunk = lambda: df[['srcip','dstip','dstport', 'x0']].groupby(["srcip", "dstip", "dstport"]).size().rename("count").reset_index()

In [13]:
%%timeit
srcdstcount_thunk()

1.68 s ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
srcdstcounts = srcdstcount_thunk()

In [15]:
csrcdstcount_thunk = lambda: cdf[['srcip','dstip','dstport', 'x0']].groupby(by=["srcip", "dstip", "dstport"], as_index=False).agg({"x0": "count"}).rename(columns={"x0": "count"})

In [16]:
%%timeit
csrcdstcount_thunk()

291 ms ± 6.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
csrcdstcounts = csrcdstcount_thunk()

In [18]:

srcdstportcount_thunk = lambda: srcdstcounts[['srcip', 'dstip', 'dstport']].drop_duplicates().groupby(['srcip', 'dstip']).size().rename("port_count").reset_index()


In [19]:
%%timeit
srcdstportcount_thunk()

900 ms ± 4.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
srcdstportcounts = srcdstportcount_thunk()

In [21]:
srcdstportcounts.sort_values(by=["port_count"], ascending=False)[:3]

,srcip,dstip,port_count
32840,192.168.204.45,192.168.23.1,40328
25585,192.168.202.79,192.168.229.254,39075
7405,192.168.202.110,192.168.28.25,38700


In [22]:
csrcdstportcount_thunk = lambda: cudf.DataFrame.from_pandas(csrcdstcounts[['srcip', 'dstip', 'dstport']].to_pandas().drop_duplicates()).groupby(by=['srcip', 'dstip'], as_index=False).agg({'dstport': "count"}).rename(columns={"dstport": "port_count"})

In [23]:
%%timeit
csrcdstportcount_thunk()

1.71 s ± 31.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
csrcdstportcounts = csrcdstportcount_thunk()

In [25]:
csrcdstportcounts.sort_values(by=["port_count"], ascending=False)[:3].to_pandas()

,srcip,dstip,port_count
32840,192.168.204.45,192.168.23.1,40328
25585,192.168.202.79,192.168.229.254,39075
7405,192.168.202.110,192.168.28.25,38700


In [ ]:
hg = graphistry.hypergraph(cdf, entity_types = ['src_ip', 'dst_ip'])
hg['graph'].plot()